# Segmenting and Clustering Neighborhoods in Toronto
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

Read page to row_data

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_postal = requests.get(url)
soup = BeautifulSoup(page_postal.content, 'html.parser')

In [3]:
# see table 1 begin tbody
table = soup.find('tbody') 


In [4]:
rows = table.select('tr')
row_data = [r.get_text() for r in rows]

In [5]:
# set dataframe from rowdata
columns=['Postcode','Borough','Neighborhood']
df = pd.DataFrame(row_data)[0].str.split('\n', expand = True)
df.drop(df.columns[[0, 4]], axis = 1, inplace=True)
df.columns = columns
df.drop(df.index[0], inplace=True) 
df.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Ignore cells with Not assigned.

In [6]:
df = pd.DataFrame(df[df.Borough != 'Not assigned'])
df.reset_index(inplace=True)
df.head(12)

,index,Postcode,Borough,Neighborhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Harbourfront
3,6,M5A,Downtown Toronto,Regent Park
4,7,M6A,North York,Lawrence Heights
5,8,M6A,North York,Lawrence Manor
6,9,M7A,Queen's Park,Not assigned
7,11,M9A,Etobicoke,Islington Avenue
8,12,M1B,Scarborough,Rouge
9,13,M1B,Scarborough,Malvern


neighborhoods've combined with same postal code                                                         

In [8]:
df_temp = df.groupby(['Postcode', 'Borough'], sort=False).agg(','.join)
df_temp.reset_index(inplace=True)

Set "Not assigned" Queen's Park

In [9]:
df  = df_temp.replace("Not assigned", "Queen's Park")
df.to_csv(r'toronto_step1.csv')
df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [10]:
#Shape
df.shape

(103, 3)